In [2]:
!pip install textract
!pip install glob2

     -------------------------------------- 106.9/106.9 kB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 32.8/32.8 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 10.1/10.1 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 69.0/69.0 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 9.7 MB/s eta 0:00:00
     -------------------------------------- 133.4/133.4 kB 7.7 MB/s eta 0:00:00
     -------------------------------------- 103.3/103.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.7/1.7 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 74.0/74.0 k

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.24 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.24 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
anaconda-client 1.11.0 requires six>=1.15.0, but you have six 1.12.0 which is incompatible.


In [1]:
#필요한 모듈 import
import re
import textract
import glob
import os

In [32]:
#전처리 함수
def PreProcess(text):
    text = re.sub(pattern='Posted on [0-9]{4} [0-9]{2} [0-9]{2}', \
                  repl='', string=text)
    text = re.sub(pattern='Posted on [0-9]{8}', \
                  repl='', string=text)
    text = re.sub(pattern='[0-9]{4}년 [0-9]{,2}월 [0-9]{,2}일 [0-9]{,2}시 [0-9]{,2}분 [0-9]{,2}초', \
                  repl='', string=text)
    text = re.sub(pattern='[0-9]{4}. [0-9]{,2}. [0-9]{,2}', \
                  repl='', string=text)
    _filter = re.compile('[\s]')
    text = _filter.sub(' ', text)
    
    _filter = re.compile('[ㄱ-ㅣ]+')
    text = _filter.sub('', text)
    
    _filter = re.compile('[\n]')
    text = _filter.sub(' ', text)
    
    _filter = re.compile('[^가-힣 0-9 a-z A-Z \. \, \' \" \? \!]+')
    text = _filter.sub('', text)
      
    text = re.sub("[.]", ".  ", text).strip()

    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

## 주의
ANSI로 인코딩된 txt 파일이어야 정상적으로 preprocessing 되는 것을 확인했음 -> 수정
utf-8로 모두 동작 가능

In [46]:
#txt file 불러오기
#num = 몇 호인지 체크하기 위함 

#사용자 지정
num = 28
end_num = 38
processed_folder_path = 'C:/Users/user/Desktop/TAVE/북커톤/data'

#while num <= end_num:
    
path = 'C:/Users/user/Desktop/TAVE/북커톤/data/literature_repreprocess.txt'
files = [f for f in glob.glob(path)]

text_list = []

for file in files:
    with open(path, 'r', encoding = 'utf8') as f:
        raw_data = f.read()
        if raw_data == '':
            continue
    text_list.append(raw_data)
    
to_train_text = ''.join(map(str, text_list))
preprocessed_text = PreProcess(to_train_text)

processed_file_path = os.path.join(processed_folder_path, "xml_병합3.txt")
with open(processed_file_path, 'w', encoding = 'utf8') as f:
        f.write(preprocessed_text) 

    #num += 1

## 맞춤법 검사기
대회 내에서는 외부 API 사용 불가로 사용하지 못함

In [62]:
import pandas as pd
import os
from hanspell import spell_checker
from tqdm import tqdm

# pp_df는 전처리 된 데이터
sc_df = pd.DataFrame({'context':[]})

data_dir = 'C:/Users/sally/Desktop/북커톤/preprocess2_대표에세이 문학회/repreprocessing'
save_dir = 'C:/Users/sally/Desktop/북커톤/preprocess2_대표에세이 문학회'

for text_name in tqdm(sorted(os.listdir(data_dir))):
    text_dir = os.path.join(data_dir, text_name)
    with open(text_dir, 'r') as f:
        raw_texts = f.readlines()
        if len(raw_texts) == 0:
            continue
        
        output_text = ''
        for raw_text in raw_texts:
            preprocessed_text = preprocessing2(raw_text)
            try:
                spell_checked_text = spell_checker.check(preprocessed_text)
                spell_checked_text = spell_checked_text.as_dict()['checked']
                if spell_checked_text and spell_checked_text[0] != ' ':
                    spell_checked_text = ' ' + spell_checked_text
            except:
                continue
            output_text = output_text + spell_checked_text
        sc_df = sc_df.append({'context': output_text}, ignore_index=True)

sc_df.head(10)
sc_df.to_csv(os.path.join(save_dir, 'essay_sc.txt'), sep='\n', header=False, index=False)
with open(os.path.join(save_dir, 'essay_sc_not_df.txt'), 'w') as f:
    for i, text in enumerate(sc_df['context']):
        if i != 0:
            f.write('\n'+text)
        else:
            f.write(text)

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sc_df = sc_df.append({'context': output_text}, ignore_index=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sc_df = sc_df.append({'context': output_text}, ignore_index=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sc_df = sc_df.append({'context': output_text}, ignore_index=True)
C:\Users\sally\AppData\Local\Temp\ipykernel_14468\3829480334.py:30: Future

In [47]:
#전처리 test

text = '齊景公問政於孔子 孔子對曰 君君 臣臣 父父 子子 公曰 善哉 信如君不君 臣不臣 父不父 子不子 雖有粟 吾得而食諸 누구나 한 번쯤 들어보았을 유명한 구절입니다.'

print(PreProcess(text))

누구나 한 번쯤 들어보았을 유명한 구절입니다.
